In [1]:
from __future__ import division
import math
import numpy as np
import torch
from torchvision.models.alexnet import alexnet
from torchvision.models.vgg import vgg16, vgg19
from torchvision.models.resnet import resnet18, resnet50
from torchvision.models.densenet import densenet121
from torchvision.models.inception import Inception3 as inception3
from torchvision import transforms
from PIL import Image
import sys
import time
import random
import cv2
import gc

sys.path.append('../../../code')

from python.commons import full_inference_e2e, inc_inference_e2e
from python.vgg16 import VGG16
from python.resnet18 import ResNet18
from python.inception3 import Inception3

from torch.autograd import Variable

import matplotlib.pyplot as plt
from plot_commons import get_ivm_patch_coordinates, calculate_ivm_flops, calculate_flops, mean_confidence_interval

random.seed(45)
np.random.seed(45)

torch.set_num_threads(8)

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def inc_inference_selected(model, file_path, patch_size, stride, patch_positions, batch_size=64, beta=1.0,
                           x0=0, y0=0, image_size=224, x_size=224, y_size=224, gpu=True, version='v1',
                           n_labels=1000, weights_data=None, loader=None, c=0.0):

    if loader == None:
        loader = transforms.Compose([transforms.Resize([image_size, image_size]), transforms.ToTensor()])
    orig_image = Image.open(file_path).convert('RGB')
    orig_image = loader(orig_image).unsqueeze(0)

    if gpu:
        orig_image = orig_image.cuda()

    image_patches = torch.FloatTensor(3, patch_size, patch_size).fill_(c).repeat(batch_size, 1, 1, 1)
        
    x_output_width = int(math.ceil((x_size*1.0 - patch_size) / stride))
    y_output_width = int(math.ceil((y_size*1.0 - patch_size) / stride))

    total_number = x_output_width * y_output_width
    logit_values = np.zeros((x_output_width, y_output_width), dtype=np.float32)
    
    num_batches = int(math.ceil(len(patch_positions) * 1.0 / batch_size))
    inc_model = model(beta=beta, gpu=gpu, n_labels=n_labels, weights_data=weights_data).eval()
    
    if gpu:
        inc_model = inc_model.cuda()
        
    temp = inc_model.forward_materialized(orig_image).cpu().data.numpy()
    logit_index = np.argmax(temp)
    prob = np.max(temp)
 
    locations = torch.zeros([batch_size, 2], dtype=torch.int32)
    
    for i in range(num_batches):
        for j in range(batch_size):
            index = j * num_batches + i
            if index >= len(patch_positions):
                break

            x, y = patch_positions[index]
            x = x*stride + x0
            y = y*stride + y0
            x,y = int(x), int(y)
            
            locations[j, 0] = x
            locations[j, 1] = y

        if version == 'v1':
            logits = inc_model.forward_gpu(image_patches, locations, p_height=patch_size, p_width=patch_size)
        else:
            logits = inc_model.forward_pytorch(image_patches, locations, p_height=patch_size, p_width=patch_size)
            
        logits = logits.cpu().data.numpy()[:, logit_index].flatten().tolist()

        for logit, j in zip(logits, range(batch_size)):
            index = j * num_batches + i
            if index >= len(patch_positions):
                break
            x, y = patch_positions[index]
            logit_values[x, y] = logit

    del inc_model
    gc.collect()

    return logit_values, prob, logit_index



def adaptive_drilldown(model, file_path, patch_size, s2, percentile, speedup, batch_size=128, image_size=224,
                       x_size=224, y_size=224, beta=1.0, gpu=True, version='v1',
                       n_labels=1000, weights_data=None, loader=None, c=0.0):

    final_out_width = int(math.ceil((image_size*1.0-patch_size)/s2))
    s1 = round(math.sqrt(speedup/(1-percentile*speedup)) * s2)
    percentile *= 100 

    #checking for interested regions
    temp1, prob, logit_index = inc_inference_e2e(model, file_path, patch_size, s1,
                                    batch_size=batch_size, beta=beta, image_size=image_size, x_size=x_size,
                              y_size=y_size, gpu=gpu, version=version, weights_data=weights_data, loader=loader, c=c)

    temp1 = cv2.resize(temp1, (final_out_width, final_out_width), interpolation=cv2.INTER_LINEAR)
    threshold = np.percentile(temp1, percentile)
    temp1 = np.where(temp1 <= threshold, 0, temp1)
    idx = np.argwhere(temp1 <= threshold)
    
    #drilldown into interested regions
    temp2, prob, logit_index = inc_inference_selected(model, file_path, patch_size, s2, idx.tolist(),
                                    batch_size=batch_size, beta=beta, image_size=image_size,
                              x_size=image_size, y_size=image_size, gpu=gpu, version=version,
                            weights_data=weights_data, loader=loader, c=c)

    
    
    return np.add(temp1, temp2), prob, logit_index

In [3]:
colors = ['blue', 'orange', 'green', 'red']

In [ ]:
models = ["VGG16", "ResNet18", "Inception3"]
patch_size = 16
S2 = 1
c = 0.0
gpu = True
drill_down_speedup = 2
drill_down_ratio = 0.25
datasets = ["oct", "chest_xray", "imagenet"]
image_file_path = '../../../code/python/dog_resized.jpg'
runs = 1

def get_tau(model, dataset):
    if model == "VGG16":
        if dataset == "chest_xray":
            return 0.5
        else:
            return 0.4
    elif model == "ResNet18":
        return 0.7
    elif model == "Inception3":
        return 0.8
    
    
fig = plt.figure(figsize=(12, 2.5))    
for idx, model in enumerate(models):
    ax = plt.subplot(1, 3, idx+1)
    
    if model == "VGG16":
        ivm_model = VGG16
        image_size = 224
    elif model == "ResNet18":
        ivm_model = ResNet18
        image_size = 224
    elif model == "Inception3":
        ivm_model = Inception3
        image_size = 299

    full_inf_mean = []
    full_inf_ci = []
    ivm_torch_mean = []
    ivm_torch_ci = []
    ivm_exact_mean = []
    ivm_exact_ci = []
    ivm_approx_mean = []
    ivm_approx_ci = []
        
        
    for dataset in datasets:
        tau = get_tau(model, dataset)

        temp_runs = []
        for count in range(runs):
            if gpu:
                torch.cuda.synchronize()
            prev_time = time.time()
            with torch.no_grad():
                x, prob, logit_index = full_inference_e2e(ivm_model, image_file_path, patch_size, S2,
                                       batch_size=128, gpu=gpu, image_size=image_size, x_size=image_size,
                                       y_size=image_size, c=c)
            if gpu:
                torch.cuda.synchronize()
            full_inference_time = time.time() - prev_time
            temp_runs.append(full_inference_time)
            print(count, model, dataset, inc_inference_time)
        gc.collect()
        m, ci = mean_confidence_interval(temp_runs)
        full_inf_mean.append(m)
        full_inf_ci.append(ci)
        
        
        temp_runs = []
        for count in range(runs):
            if gpu:
                torch.cuda.synchronize()    
            prev_time = time.time()
            with torch.no_grad():
                x, prob, logit_index = inc_inference_e2e(ivm_model, image_file_path, patch_size, S2,
                                      batch_size=256, beta=1.0, gpu=gpu, version='v2',
                                     image_size=image_size, x_size=image_size, y_size=image_size, c=c)
            if gpu:
                torch.cuda.synchronize()
            inc_inference_time = time.time() - prev_time
            temp_runs.append(inc_inference_time)
            print(count, model, dataset, inc_inference_time)
            
        gc.collect()
        m, ci = mean_confidence_interval(temp_runs)
        ivm_torch_mean.append(m)
        ivm_torch_ci.append(ci)
        
        temp_runs = []
        for count in range(runs):
            if gpu:
                torch.cuda.synchronize()    
            prev_time = time.time()
            with torch.no_grad():
                x, prob, logit_index = inc_inference_e2e(ivm_model, image_file_path, patch_size, S2,
                                      batch_size=256, beta=1.0, gpu=gpu, version='v1',
                                     image_size=image_size, x_size=image_size, y_size=image_size, c=c)
            if gpu:
                torch.cuda.synchronize()
            inc_inference_time = time.time() - prev_time
            temp_runs.append(inc_inference_time)
            print(count, model, dataset, inc_inference_time)
        m, ci = mean_confidence_interval(temp_runs)
        ivm_exact_mean.append(m)
        ivm_exact_ci.append(ci)
        
        
        temp_runs = []
        for count in range(runs):
            if gpu:
                torch.cuda.synchronize()    
            prev_time = time.time()
            with torch.no_grad():
                x, prob, logit_index = adaptive_drilldown(ivm_model, image_file_path, patch_size, S2, 0.25, 2,
                                    batch_size=256, beta=tau, gpu=gpu, version='v1',
                                    image_size=image_size, x_size=image_size, y_size=image_size,
                                    c=c)
            if gpu:
                torch.cuda.synchronize()
            ada_time = time.time() - prev_time
            temp_runs.append(ada_time)
            print(count, model, dataset, ada_time)
        gc.collect()
        m, ci = mean_confidence_interval(temp_runs)
        ivm_approx_mean.append(m)
        ivm_approx_ci.append(ci)
        
        
    pos = [0.8, 2., 3.2]
    width = 0.2

    l1 = plt.bar(pos, full_inf_mean, width, color=colors[0], hatch=3*'-',
                 yerr=full_inf_ci, ecolor='black', alpha=0.5) 
    l2 = plt.bar([p + width for p in pos], ivm_torch_mean, width, color=colors[1], hatch=3*'+',
             yerr=ivm_torch_ci, ecolor='black', alpha=0.5)
    l3 = plt.bar([p + 2*width for p in pos], ivm_exact_mean, width, color=colors[2], hatch=3*'+',
             yerr=ivm_exact_ci, ecolor='black', alpha=0.5)
    l4 = plt.bar([p + 3*width for p in pos], ivm_approx_mean, width, color=colors[3], hatch=3*'+',
             yerr=ivm_approx_ci, ecolor='black', alpha=0.5)
    
    if idx == 0:
        ax.set_ylabel('Run Time (s)')
    ax.set_xticks([p + 1.5*width for p in pos])
    ax.set_title(model+"/GPU")
    ax.set_xticklabels(['OCT', 'Chest X-Ray', 'ImageNet'])
    plt.xlim(min(pos)-width, max(pos)+width*4)
    plt.grid()
    
    
    
    
#     gpu=False
    
#     ax = plt.subplot(2, 3, idx+1)

    
#     full_inf_mean = []
#     full_inf_ci = []
#     ivm_exact_mean = []
#     ivm_exact_ci = []
#     ivm_approx_mean = []
#     ivm_approx_ci = []
        
        
#     for dataset in datasets:
#         tau = get_tau(model, dataset)

#         temp_runs = []
#         for count in range(runs):
#             if gpu:
#                 torch.cuda.synchronize()
#             prev_time = time.time()
#             with torch.no_grad():
#                 x, prob, logit_index = full_inference_e2e(ivm_model, image_file_path, patch_size, S2,
#                                        batch_size=16, gpu=gpu, image_size=image_size, x_size=image_size,
#                                        y_size=image_size, c=c)
#             if gpu:
#                 torch.cuda.synchronize()
#             full_inference_time = time.time() - prev_time
#             temp_runs.append(full_inference_time)
#         m, ci = mean_confidence_interval(temp_runs)
#         full_inf_mean.append(m)
#         full_inf_ci.append(ci)
        
        
#         temp_runs = []
#         for count in range(runs):
#             if gpu:
#                 torch.cuda.synchronize()    
#             prev_time = time.time()
#             with torch.no_grad():
#                 x, prob, logit_index = inc_inference_e2e(ivm_model, image_file_path, patch_size, S2,
#                                       batch_size=16, beta=1.0, gpu=gpu, version='v2',
#                                      image_size=image_size, x_size=image_size, y_size=image_size, c=c)
#             if gpu:
#                 torch.cuda.synchronize()
#             inc_inference_time = time.time() - prev_time
#             temp_runs.append(full_inference_time)
#         m, ci = mean_confidence_interval(temp_runs)
#         ivm_exact_mean.append(m)
#         ivm_exact_ci.append(ci)
        
        
#         temp_runs = []
#         for count in range(runs):
#             if gpu:
#                 torch.cuda.synchronize()    
#             prev_time = time.time()
#             with torch.no_grad():
#                 x, prob, logit_index = adaptive_drilldown(ivm_model, image_file_path, patch_size, S2, 0.25, 2,
#                                     batch_size=16, beta=tau, gpu=gpu, version='v2',
#                                     image_size=image_size, x_size=image_size, y_size=image_size,
#                                     c=c)
#             if gpu:
#                 torch.cuda.synchronize()
#             ada_time = time.time() - prev_time
#             temp_runs.append(ada_time)
#         m, ci = mean_confidence_interval(temp_runs)
#         ivm_approx_mean.append(m)
#         ivm_approx_ci.append(ci)
        
        
#     pos = [0.8, 2., 3.2]
#     width = 0.2

#     l1 = plt.bar(pos, full_inf_mean, width, color=colors[0], hatch=3*'-',
#                  yerr=full_inf_ci, ecolor='black', alpha=0.5) 
#     l3 = plt.bar([p + width for p in pos], ivm_exact_mean, width, color=colors[2], hatch=3*'+',
#              yerr=ivm_exact_ci, ecolor='black', alpha=0.5)
#     l4 = plt.bar([p + 2*width for p in pos], ivm_approx_mean, width, color=colors[3], hatch=3*'+',
#              yerr=ivm_approx_ci, ecolor='black', alpha=0.5)
    
#     ax.set_ylabel('Run Time (s)')
#     ax.set_xticks([p + width for p in pos])
#     ax.set_title(model+"/CPU")
#     ax.set_xticklabels(['OCT', 'Chest X-Ray', 'ImageNet'])
#     plt.xlim(min(pos)-width, max(pos)+width*3)
#     plt.grid()
        
   

lgd = fig.legend((l1, l2, l3, l4), ('Naive', 'Naive Inc. Inference-Exact', 'Krypton-Exact', 'Krypton-Approx.'), loc=(0.25, -0.01), ncol=4, frameon=False)

plt.savefig('../images/5_1.pdf', bbox_extra_artists=(lgd,), bbox_inches='tight')   

plt.show()